# Read the tables, we have 1 per model 
# Separate the first n line and the next n and so for each num of labeled samples 
# combine
# write to csv

In [1]:
import pandas as pd 

# Location of the results csv files
results_prefix_str = 'FP2_SP_sameWind_1layer_unfrozen/s3l_'
#results_prefix_str = 'XGB/s3l_'
#results_prefix_str = 'baseline_sameWind_1layer/s3l_'

#model_list = ['xgb', 'dae_xgb', 'scarf_xgb', 'vime_xgb' ,'subtab_xgb', 'switchtab_xgb']
#model_list = ['3layer_mlp', '4layer_mlp']
model_list = ['dae', 'scarf', 'vime']
#model_list = ['1layer_mse_mlp', '1layer_l1loss_mlp','1layer_smoothl1loss_mlp','1layer_mape_mlp']

#num_samples_list = ['100', '1K', '10K', '20K']
num_samples_list = ['100', '250', '500', '1K', '10K', '20K']

#tasks = ['VR fragment thput.', 'VR burst thput.', 'VR fragment latency', 'VR burst latency', 
#         'UL probe latency', 
#         'DL probe latency'
#        ]
tasks = ['T 1', 'T 2', 'T 3', 'T 4', 'T 5']
task_str = ["\\textbf{T 1} & ", 
                "\\textbf{T 2} & ", 
                "\\textbf{T 3} & ", 
                "\\textbf{T 4} & ", 
                "\\textbf{T 5} & "
               ]

results = {}
# Read all the results
for model in model_list:
    results_ff = pd.read_csv(results_prefix_str+model+'_test_results.csv', delimiter=",", header=None)
    results[model] = {}
    i_start = 0
    for samp in num_samples_list:
        results[model][samp] = results_ff.iloc[i_start:i_start+5]
        i_start = i_start + 5
    
# For each error metric type 
# for each num of samples 
for num_samp in num_samples_list:
    print('\hline')
    print(r"\multicolumn{"+str(len(model_list)+1)+r"}{|c|}{\rule{0pt}{2.5ex}Num. of labeled samples = "+num_samp+r"\rule[-1ex]{0pt}{2.5ex}}\\")
    print('\hline')
    #print('Num of samples: ', num_samp)
    # create 3 err metric dataframnes 
    err_r2 = pd.DataFrame(index=tasks, columns=model_list)
    err_mape = pd.DataFrame(index=tasks, columns=model_list)
    err_mae = pd.DataFrame(index=tasks, columns=model_list)
    err_r2 = err_r2.astype(float)
    err_mape = err_mape.astype(float)
    err_mae = err_mae.astype(float)
    
    for model in model_list:
        # add one column at a time 
        err_mae.loc[:, model] = results[model][num_samp].iloc[:,1]
        err_mape.loc[:, model] = results[model][num_samp].iloc[:,2]
        err_r2.loc[:, model] = results[model][num_samp].iloc[:,3]
        #print('model:', model)
        #print(results[model][num_samp])
    
    
    #print(err_mae)
    #print(err_mape)
    #print(err_r2)
    err_r2 = err_r2.round(3)
    err_mae = err_mae.round(3)
    err_mape = err_mape.round(3)

    # CHECK HERE !!!!
    error_metric_to_print = err_r2

    # Convert DataFrame to a LaTeX-friendly format
    latex_table = error_metric_to_print.apply(lambda row: ' & '.join(row.astype(str)), axis=1)
    latex_table = task_str + latex_table + ' \\\\'
    
    # Join the rows with newline characters
    latex_table_str = '\n'.join(latex_table)
    
    print(latex_table_str)
    
# Print with & as the delimiter to make it each to import into latex 

FileNotFoundError: [Errno 2] No such file or directory: 'FP2_SP_sameWind_1layer_unfrozen/s3l_dae_test_results.csv'